In [10]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
import pickle
import mne
import re

from mne.channels import make_1020_channel_selections
from mne.event import define_target_events
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm


from sklearn.model_selection import KFold, train_test_split, cross_validate
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import warnings
warnings.filterwarnings("ignore")


from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

class_names = ['1','2','3']

In [2]:
os.chdir('/data/castrogaray-j/Spectrogram_images')
files = os.listdir()
# pkl_files = sorted([file for file in files if file.startswith('raw') and file.endswith('.pkl')])
# subjects = set([file[4:6] for file in pkl_files])
# #pkl_files
# len(pkl_files)
# int(pkl_files[9][8:10])
print(len(files))
print(files[0])
print(files[0][8:10], files[0][17], files[100][33:36])



586368
subject_18_class_1_label_0_event_165_channel_P2.png
18 1 24_


In [32]:
def get_tags(file):
    

    pattern = r"subject_(\d+)_class_(\d+)_label_\d+_event_(\d+)_channel_"
    subject, class_tag, event = re.findall(pattern, file)[0]
    

    

    return  class_tag, subject, event
    

In [15]:
pattern = r"subject_(\d+)_class_(\d+)_label_\d+_event_(\d+)_channel_"
matches = re.findall(pattern, files[10])

for match in matches:
    subject, class_, event = match
    print(f"Subject: {subject}, Class: {class_}, Event: {event}")

Subject: 9, Class: 3, Event: 56


In [16]:
os.chdir('/home/castrogaray-j/U_Winnipeg_OneDrive/Pattern recognition/BICT/raw')
with open("raw_performance_percentiles_2.pkl", 'rb') as file:
        df = pickle.load(file)

df = df.iloc[:,64:]
df

,class,subject,label,drop 10,drop 20,event
0,1,1,0,0.0,0.0,1
1,1,1,0,0.0,0.0,1
2,1,1,0,0.0,0.0,1
3,1,1,0,0.0,0.0,1
4,1,1,0,0.0,0.0,1
...,...,...,...,...,...,...
46918597,3,21,1,1.0,1.0,164
46918598,3,21,1,1.0,1.0,164
46918599,3,21,1,1.0,1.0,164
46918600,3,21,1,1.0,1.0,164


In [35]:
df1 = df.drop_duplicates(subset=['class', 'subject', 'event'])
df1.reset_index(drop=True, inplace=True)
df_files = pd.DataFrame(files)
df_files

,0
0,subject_18_class_1_label_0_event_165_channel_P...
1,subject_14_class_1_label_0_event_160_channel_P...
2,subject_17_class_3_label_0_event_143_channel_P...
3,subject_17_class_3_label_0_event_147_channel_F...
4,subject_14_class_1_label_1_event_85_channel_C2...
...,...
586363,subject_11_class_3_label_1_event_83_channel_P9...
586364,subject_1_class_2_label_1_event_107_channel_F8...
586365,subject_2_class_3_label_1_event_124_channel_FC...
586366,subject_18_class_1_label_1_event_6_channel_Iz.png


In [ ]:

# Aplicar la función get_tags a cada archivo en df_files
#for i in range(0, len(df_files)):
    #df_files.loc[i, 'class'], df_files.loc[i, 'subject'], df_files.loc[i, 'event'] = get_tags(df_files[0][i])
df_files[['class', 'subject', 'event']] = df_files[0].apply(lambda x: pd.Series(get_tags(x)))

# Convertir las columnas a tipo int para que coincidan con df1
df_files['class'] = df_files['class'].astype(int)
df_files['subject'] = df_files['subject'].astype(int)
df_files['event'] = df_files['event'].astype(int)


df_files


,0,class,subject,event
0,subject_18_class_1_label_0_event_165_channel_P...,1,18,165
1,subject_14_class_1_label_0_event_160_channel_P...,1,14,160
2,subject_17_class_3_label_0_event_143_channel_P...,3,17,143
3,subject_17_class_3_label_0_event_147_channel_F...,3,17,147
4,subject_14_class_1_label_1_event_85_channel_C2...,1,14,85
...,...,...,...,...
586363,subject_11_class_3_label_1_event_83_channel_P9...,3,11,83
586364,subject_1_class_2_label_1_event_107_channel_F8...,2,1,107
586365,subject_2_class_3_label_1_event_124_channel_FC...,3,2,124
586366,subject_18_class_1_label_1_event_6_channel_Iz.png,1,18,6


In [43]:
df_joined = df1.merge(df_files, on=['class', 'subject', 'event'], how='right')
df_joined.rename(columns={0: 'path'}, inplace=True)
df_joined['path'] = '/data/castrogaray-j/Spectrogram_images/' + df_joined['path'].astype(str)
df_joined

,class,subject,label,drop 10,drop 20,event,path
0,1,18,0.0,0.0,0.0,165,/data/castrogaray-j/Spectrogram_images/subject...
1,1,14,0.0,NaN,NaN,160,/data/castrogaray-j/Spectrogram_images/subject...
2,3,17,0.0,NaN,NaN,143,/data/castrogaray-j/Spectrogram_images/subject...
3,3,17,0.0,0.0,0.0,147,/data/castrogaray-j/Spectrogram_images/subject...
4,1,14,0.0,0.0,NaN,85,/data/castrogaray-j/Spectrogram_images/subject...
...,...,...,...,...,...,...,...
586363,3,11,1.0,1.0,1.0,83,/data/castrogaray-j/Spectrogram_images/subject...
586364,2,1,1.0,1.0,1.0,107,/data/castrogaray-j/Spectrogram_images/subject...
586365,3,2,1.0,1.0,1.0,124,/data/castrogaray-j/Spectrogram_images/subject...
586366,1,18,1.0,1.0,1.0,6,/data/castrogaray-j/Spectrogram_images/subject...


In [45]:
df_joined.to_csv('/home/castrogaray-j/U_Winnipeg_OneDrive/Digital_image_processing/BCIT_spectrogram/paths_with_labels.csv', index=False)